In [40]:
import psycopg2
import pandas as pd
from pathlib import Path
import csv


conn = psycopg2.connect(database="f1_halo",
                       host="localhost",
                       user="postgres",
                       password = "postgres",
                       port="5432")

In [41]:
# Store filepaths into variable
driver_csv = Path("data/drivers.csv")
fcyphases_csv = Path("data/fcyphases.csv")
laps_csv = Path("data/laps.csv")
qualifyings_csv = Path("data/qualifyings.csv")
races_csv = Path("data/races.csv")
retirements_csv = Path("data/retirements.csv")
starterfields_csv = Path("data/starterfields.csv")
weather = Path("data/merged_weather.csv")
weather_long = Path("data/f1_weather_long.csv")
weather_df = pd.read_csv(weather)
weather_long_df = pd.read_csv(weather_long)

In [42]:
tables = ["drivers","fcyphases","laps","qualifyings","races","retirements","starterfields"]
for i in tables:
    cursor = conn.cursor()
    query = f'SELECT * from {i};'
    #print(query)
    cursor.execute(query)
    rows = cursor.fetchall()
    df_name = i + "_df"
    globals()[df_name]= pd.DataFrame(rows)
    print(df_name)
    
    
    #csv for headers
    path = f'data/{i}.csv'
    csv_path = Path(path)
    with open(csv_path, 'r', newline='') as file:
        reader = csv.reader(file)
        headers = next(reader)  # Read the first row
        if len(globals()[df_name].columns) == len(headers):
            globals()[df_name].columns = headers

    #print(df_name)
    print(globals()[df_name].columns)

drivers_df
Index(['id', 'carno', 'initials', 'name'], dtype='object')
fcyphases_df
Index(['id', 'race_id', 'startracetime', 'endracetime', 'startraceprog',
       'endraceprog', 'startlap', 'endlap', 'type'],
      dtype='object')
laps_df
Index(['race_id', 'lapno', 'position', 'driver_id', 'laptime', 'racetime',
       'gap', 'interval', 'compound', 'tireage', 'pitintime',
       'pitstopduration', 'nextcompound', 'startlapprog_vsc', 'endlapprog_vsc',
       'age_vsc', 'startlapprog_sc', 'endlapprog_sc', 'age_sc'],
      dtype='object')
qualifyings_df
Index(['race_id', 'position', 'driver_id', 'q1laptime', 'q2laptime',
       'q3laptime', 'speedtrap'],
      dtype='object')
races_df
Index(['id', 'date', 'season', 'location', 'availablecompounds', 'comment',
       'nolaps', 'nolapsplanned', 'tracklength'],
      dtype='object')
retirements_df
Index(['season', 'driver_id', 'accidents', 'failures'], dtype='object')
starterfields_df
Index(['race_id', 'driver_id', 'team', 'teamcolor', 'eng

In [43]:
# make column name variables constant throughout all dfs
drivers_df.rename(columns = {'id':'driver_id'}, inplace = True)
# drivers_df.to_csv('drivers.csv') only do this if create a seperate data file
races_df.rename(columns = {'id':'race_id'}, inplace = True)

In [44]:
# make adjustments to column names to better represent data 

# for "retirements" df... NOTE: accidents and failures is per driver per season (not per race)
retirements_df.rename(columns = {'accidents':'accidents per driver, per season'}, inplace = True)
retirements_df.rename(columns = {'failures':'failures per driver, per season'}, inplace = True)

#for weather_df...
weather_df.rename(columns = {'constructor': 'team'}, inplace = True)
weather_df.rename(columns = {'length': 'tracklength'}, inplace = True)
# Convert the 'date' column to datetime format
weather_df['date'] = pd.to_datetime(weather_df['date'], format='%m/%d/%Y')
# Convert the 'date' column to the desired format with dashes
weather_df['date'] = weather_df['date'].dt.strftime('%Y-%m-%d')

# for weather_long_df, convert date and change column names
weather_long_df.rename(columns = {'description': 'weather expanded'}, inplace = True)
weather_long_df.rename(columns = {'temp': 'temperature'}, inplace = True)
weather_long_df.rename(columns = {'main': 'weather in brief'}, inplace = True)
# Convert the 'date' column to datetime format
weather_long_df['date'] = pd.to_datetime(weather_long_df['date'], format='%m/%d/%Y')
# Convert the 'date' column to the desired format with dashes
weather_long_df['date'] = weather_long_df['date'].dt.strftime('%Y-%m-%d')
weather_long_df

,id,temperature,feels_like,humidity,clouds,wind_speed,weather in brief,weather expanded,icon,date
0,1,65,63,38,100,10.30,Rain,Light Rain,10d,2014-03-16
1,2,89,98,62,75,4.63,Clouds,Broken Clouds,04d,2014-03-30
2,3,72,72,64,0,5.87,Clear,Clear Sky,01n,2014-04-06
3,4,64,63,67,75,3.00,Clouds,Broken Clouds,04d,2014-04-20
4,5,74,74,53,40,7.20,Clouds,Scattered Clouds,03d,2014-05-11
...,...,...,...,...,...,...,...,...,...,...
116,117,74,74,52,0,8.20,Clear,Clear Sky,01d,2019-10-13
117,118,73,72,46,75,3.10,Rain,Light Rain,10d,2019-10-27
118,119,72,70,24,20,4.60,Clouds,Few Clouds,02d,2019-11-03
119,120,68,68,64,40,7.20,Clouds,Scattered Clouds,03d,2019-11-17


In [53]:
# make adjustments to data sets to make more concise/eliminate redundancies

# refine weather_long_df by dropping coluns and prepping for merge with weather_df
#weather_long_df = weather_long_df.drop(columns = ['id','feels_like','icon'])

# make adjustments to data sets to make more concise/eliminate redundancies
weather_df_unique_weather = weather_df.drop_duplicates(subset=['date'])
    #weather_df_unique_weather = weather_df_unique_weather.set_index(['season','date'])
    #weather_df_unique_weather.sort_index()

# MERGE WEATHER DATA
weather_inclusive = pd.merge(weather_long_df,weather_df_unique_weather)

# save large weather dataset to weather_df_inclusive csv
weather_inclusive.to_csv('data/weather_df_inclusive.csv')
weather_inclusive

# refine weather df
weather_df_unique_weather_refined = weather_inclusive.drop(columns=['distance','race_name','circuitId','driver_name','time','dateOfBirth','type','direction','tracklength','nationality'])
weather_df_unique_weather_refined.sort_index()

# save medium weather dataset to weather_df_inclusive_refined csv
weather_df_unique_weather_refined.to_csv('data/weather_df_inclusive_refined.csv')


# refine weather_df_unique_weather_refined weather df further
weather_df_unique_weather_refined2 = weather_df_unique_weather_refined.drop(columns=['Unnamed: 0','round','name','team','initials','finish_position','grid','points','qual_position','q_best', 'q_worst','q_mean', 'status'])
weather_df_unique_weather_refined2.sort_index()

# save refined weather dataset to weather_df_refined csv
weather_df_unique_weather_refined2.to_csv('data/weather_df_refined.csv')
#weather_df_unique_weather_refined2.set_index(['season','date'])
#weather_df_unique_weather_refined2.sort_index()

In [54]:
merge1 = pd.merge(drivers_df, starterfields_df)

In [55]:
merge2 = pd.merge(merge1, races_df)
merge2

,driver_id,carno,initials,name,race_id,team,teamcolor,enginemanufacturer,gridposition,status,...,completedlaps,speedtrap,date,season,location,availablecompounds,comment,nolaps,nolapsplanned,tracklength
0,1,44,HAM,Lewis Hamilton,1,Mercedes,#00D2BE,Mercedes,1,DNF,...,2,252.8,2014-03-16,2014,Melbourne,"A2,A3,I,W",null,57,58,5303
1,2,3,RIC,Daniel Ricciardo,1,RedBull,#1E41FF,Renault,2,DQ,...,57,292.7,2014-03-16,2014,Melbourne,"A2,A3,I,W",null,57,58,5303
2,3,6,ROS,Nico Rosberg,1,Mercedes,#00D2BE,Mercedes,3,F,...,57,299.1,2014-03-16,2014,Melbourne,"A2,A3,I,W",null,57,58,5303
3,4,20,MAG,Kevin Magnussen,1,McLaren,#FF8700,Mercedes,4,F,...,57,316.9,2014-03-16,2014,Melbourne,"A2,A3,I,W",null,57,58,5303
4,5,14,ALO,Fernando Alonso,1,Ferrari,#DC0000,Ferrari,5,F,...,57,304.5,2014-03-16,2014,Melbourne,"A2,A3,I,W",null,57,58,5303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2474,40,16,LEC,Charles Leclerc,121,Ferrari,#DC0000,Ferrari,3,F,...,55,313.4,2019-12-01,2019,YasMarina,"A4,A6,A7,I,W",null,55,55,5554
2475,42,4,NOR,Lando Norris,121,McLaren,#FF8700,Renault,6,F,...,54,330.4,2019-12-01,2019,YasMarina,"A4,A6,A7,I,W",null,55,55,5554
2476,43,23,ALB,Alexander Albon,121,RedBull,#1E41FF,Honda,5,F,...,55,330.7,2019-12-01,2019,YasMarina,"A4,A6,A7,I,W",null,55,55,5554
2477,44,63,RUS,George Russell,121,Williams,#192c4e,Mercedes,18,F,...,54,329.3,2019-12-01,2019,YasMarina,"A4,A6,A7,I,W",null,55,55,5554


In [56]:
merge3 = pd.merge(merge2, retirements_df)
merge3

,driver_id,carno,initials,name,race_id,team,teamcolor,enginemanufacturer,gridposition,status,...,date,season,location,availablecompounds,comment,nolaps,nolapsplanned,tracklength,"accidents per driver, per season","failures per driver, per season"
0,1,44,HAM,Lewis Hamilton,1,Mercedes,#00D2BE,Mercedes,1,DNF,...,2014-03-16,2014,Melbourne,"A2,A3,I,W",null,57,58,5303,0,3
1,1,44,HAM,Lewis Hamilton,2,Mercedes,#00D2BE,Mercedes,1,F,...,2014-03-30,2014,KualaLumpur,"A1,A2,I,W",null,56,56,5543,0,3
2,1,44,HAM,Lewis Hamilton,3,Mercedes,#00D2BE,Mercedes,2,F,...,2014-04-06,2014,Sakhir,"A2,A3,I,W","Crash GUT in lap 41, SC somewhen in the end of...",57,57,5412,0,3
3,1,44,HAM,Lewis Hamilton,4,Mercedes,#00D2BE,Mercedes,1,F,...,2014-04-20,2014,Shanghai,"A2,A3,I,W",null,54,56,5451,0,3
4,1,44,HAM,Lewis Hamilton,5,Mercedes,#00D2BE,Mercedes,1,F,...,2014-05-11,2014,Catalunya,"A1,A2,I,W",null,66,66,4655,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2474,45,88,KUB,Robert Kubica,117,Williams,#192c4e,Mercedes,20,F,...,2019-10-13,2019,Suzuka,"A2,A3,A4,I,W",null,52,53,5807,0,2
2475,45,88,KUB,Robert Kubica,118,Williams,#192c4e,Mercedes,20,F,...,2019-10-27,2019,MexicoCity,"A3,A4,A6,I,W",null,71,71,4304,0,2
2476,45,88,KUB,Robert Kubica,119,Williams,#192c4e,Mercedes,19,DNF,...,2019-11-03,2019,Austin,"A3,A4,A6,I,W",null,56,56,5513,0,2
2477,45,88,KUB,Robert Kubica,120,Williams,#192c4e,Mercedes,19,F,...,2019-11-17,2019,SaoPaulo,"A2,A3,A4,I,W",null,71,71,4309,0,2


In [ ]:
meta_data = pd.merge(weather_df_unique_weather_refined2, merge3)
meta_data

In [ ]:
meta_data_refined = meta_data[['season', 'date', 'race_id','location','circuitName','weather','lat','long','driver_id','name','team','enginemanufacturer','resultposition','gridposition','status','completedlaps','comment','nolaps','tracklength']]
meta_data_refined = meta_data_refined.set_index(['season', 'date'])
meta_data_refined.sort_index()

In [ ]:
meta_data.set_index(['season','date','location','circuitName'])

In [ ]:
# save meta data frame to files
meta_data_refined.to_csv('data/meta_data_refined.csv')

In [ ]:
## creating refined weather data sets filtered on year for each track location

weather_df_unique_weather_refined2 = weather_df_unique_weather_refined2.set_index(['season','date'])

#2014
weather_df_2014 = weather_df_unique_weather_refined2[weather_df_unique_weather_refined2.index.get_level_values('season') == 2014]
weather_df_2014.sort_index()
weather_df_2014.to_csv('data/weathermap/weather_df_2014.csv')

#2015
weather_df_2015 = weather_df_unique_weather_refined2[weather_df_unique_weather_refined2.index.get_level_values('season') == 2015]
weather_df_2015.sort_index()
weather_df_2015.to_csv('data/weathermap/weather_df_2015.csv')

#2016
weather_df_2016 = weather_df_unique_weather_refined2[weather_df_unique_weather_refined2.index.get_level_values('season') == 2016]
weather_df_2016.sort_index()
weather_df_2016.to_csv('data/weathermap/weather_df_2016.csv')

#2017
weather_df_2017 = weather_df_unique_weather_refined2[weather_df_unique_weather_refined2.index.get_level_values('season') == 2017]
weather_df_2017.sort_index()
weather_df_2017.to_csv('data/weathermap/weather_df_2017.csv')

#2018
weather_df_2018 = weather_df_unique_weather_refined2[weather_df_unique_weather_refined2.index.get_level_values('season') == 2018]
weather_df_2018.sort_index()
weather_df_2018.to_csv('data/weathermap/weather_df_2018.csv')

#2019
weather_df_2019 = weather_df_unique_weather_refined2[weather_df_unique_weather_refined2.index.get_level_values('season') == 2019]
weather_df_2019.sort_index()
weather_df_2019.to_csv('data/weathermap/weather_df_2019.csv')


In [ ]:
## creating refined incident map data sets filtered on year for each track location
    # first refine incident data frame from meta data
incident_report_df =  meta_data[['season', 'date', 'race_id','location','circuitName','weather','lat','long','name','status', 'accidents per driver, per season','failures per driver, per season']]
incident_report_df.sort_index()
incident_report_df.to_csv('data/incidentmap/incident_report_df_INCLUSIVE.csv')
# refine the incident data per location per year
incident_report_df_refined = incident_report_df[['season', 'date', 'race_id','location','circuitName','weather','lat','long','name','status']]
incident_report_df_refined = incident_report_df_refined.set_index(['season', 'date','location'])
incident_report_df_refined.sort_index()
incident_report_df_refined.to_csv('data/incidentmap/incident_report_df_REFINED.csv')


#2014
incident_df_2014 = incident_report_df_refined[incident_report_df_refined.index.get_level_values('season') == 2014]
incident_df_2014.sort_index()
incident_df_2014.to_csv('data/incidentmap/incident_df_2014.csv')

#2015
weather_df_2015 = incident_report_df_refined[incident_report_df_refined.index.get_level_values('season') == 2015]
weather_df_2015.sort_index()
weather_df_2015.to_csv('data/incidentmap/incident_df_2015.csv')

#2016
weather_df_2016 = incident_report_df_refined[incident_report_df_refined.index.get_level_values('season') == 2016]
weather_df_2016.sort_index()
weather_df_2016.to_csv('data/incidentmap/incident_df_2016.csv')

#2017
weather_df_2017 = incident_report_df_refined[incident_report_df_refined.index.get_level_values('season') == 2017]
weather_df_2017.sort_index()
weather_df_2017.to_csv('data/incidentmap/incident_df_2017.csv')

#2018
weather_df_2018 = incident_report_df_refined[incident_report_df_refined.index.get_level_values('season') == 2018]
weather_df_2018.sort_index()
weather_df_2018.to_csv('data/incidentmap/incident_df_2018.csv')

#2019
weather_df_2019 = incident_report_df_refined[incident_report_df_refined.index.get_level_values('season') == 2019]
weather_df_2019.sort_index()
weather_df_2019.to_csv('data/incidentmap/incident_df_2019.csv')

In [ ]:
## creating refined driver race result data sets filtered on year for each driver
    # first refine data frame from meta data
driver_report_df =  meta_data[['season', 'date', 'race_id','location','circuitName','weather','name','team','enginemanufacturer','status', 'accidents per driver, per season','failures per driver, per season']]
driver_report_df.sort_index()
driver_report_df.to_csv('data/driverdata/driver_report_df_INCLUSIVE.csv')
# refine the driver data per driver per year
driver_report_df_refined = driver_report_df[['season', 'name','team','enginemanufacturer','accidents per driver, per season','failures per driver, per season']]
driver_report_df_refined = driver_report_df_refined.set_index(['season', 'name'])
driver_report_df_refined.sort_index()
driver_report_df_refined.to_csv('data/driverdata/driver_report_df_REFINED.csv')


#2014
driver_report_df_2014 = driver_report_df_refined[driver_report_df_refined.index.get_level_values('season') == 2014]
driver_report_df_2014.sort_index()
driver_report_df_2014.to_csv('data/driverdata/driver_report_df_2014.csv')

#2015
driver_report_df_2015 = driver_report_df_refined[driver_report_df_refined.index.get_level_values('season') == 2014]
driver_report_df_2015.sort_index()
driver_report_df_2015.to_csv('data/driverdata/driver_report_df_2015.csv')

#2016
driver_report_df_2016 = driver_report_df_refined[driver_report_df_refined.index.get_level_values('season') == 2014]
driver_report_df_2016.sort_index()
driver_report_df_2016.to_csv('data/driverdata/driver_report_df_2016.csv')

#2017
driver_report_df_2017 = driver_report_df_refined[driver_report_df_refined.index.get_level_values('season') == 2014]
driver_report_df_2017.sort_index()
driver_report_df_2017.to_csv('data/driverdata/driver_report_df_2017.csv')

#2018
driver_report_df_2018 = driver_report_df_refined[driver_report_df_refined.index.get_level_values('season') == 2014]
driver_report_df_2018.sort_index()
driver_report_df_2018.to_csv('data/driverdata/driver_report_df_2018.csv')

#2019
driver_report_df_2019 = driver_report_df_refined[driver_report_df_refined.index.get_level_values('season') == 2014]
driver_report_df_2019.sort_index()
driver_report_df_2019.to_csv('data/driverdata/driver_report_df_2019.csv')  






## creating driver report data personal to driver filtered on year (season)
driver_report_df_refined_perdriver = meta_data[['name','season','team','enginemanufacturer','accidents per driver, per season','failures per driver, per season']]
driver_report_df_refined_perdriver = driver_report_df_refined_perdriver.set_index(['name'])
    #drop season duplicates so results are per season
driver_report_df_refined_perdriver = driver_report_df_refined_perdriver.drop_duplicates(subset=['season'])
driver_report_df_refined_perdriver.sort_index()
driver_report_df_refined_perdriver.to_csv('data/driverdata/driver_report_df_REFINED_perdriver.csv')

##LewisHam
driver_report_df_Hamilton = driver_report_df_refined_perdriver[driver_report_df_refined_perdriver.index.get_level_values('name') == 'Lewis Hamilton']
driver_report_df_Hamilton.sort_index()
driver_report_df_Hamilton.to_csv('data/driverdata/driver_report_df_Hamilton.csv')

##DaniRic
driver_report_df_Ricciardo = driver_report_df_refined_perdriver[driver_report_df_refined_perdriver.index.get_level_values('name') == 'Daniel Ricciardo']
driver_report_df_Ricciardo.sort_index()
driver_report_df_Ricciardo.to_csv('data/driverdata/driver_report_df_Ricciardo.csv')

##FernAlon
driver_report_df_Alonso = driver_report_df_refined_perdriver[driver_report_df_refined_perdriver.index.get_level_values('name') == 'Fernando Alonso']
driver_report_df_Alonso.sort_index()
driver_report_df_Alonso.to_csv('data/driverdata/driver_report_df_Alonso.csv')

##Leclerc
driver_report_df_Leclerc = driver_report_df_refined_perdriver[driver_report_df_refined_perdriver.index.get_level_values('name') == 'Charles Leclerc']
driver_report_df_Leclerc.sort_index()
driver_report_df_Leclerc.to_csv('data/driverdata/driver_report_df_Leclerc.csv')

##GRussell
driver_report_df_Russell = driver_report_df_refined_perdriver[driver_report_df_refined_perdriver.index.get_level_values('name') == 'George Russell']
driver_report_df_Russell.sort_index()
driver_report_df_Russell.to_csv('data/driverdata/driver_report_df_Russell.csv')


In [ ]:
# save meta data frame to files
meta_data = meta_data.set_index(['season','date','location','circuitName'])
meta_data.to_csv('meta_data_inclusive.csv')